In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import cv2 as cv
import os
from PIL import Image

In [30]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [95]:
efficientnet = models.efficientnet_b0(pretrained = True)
num_features = efficientnet.classifier[1].in_features
efficientnet.classifier[1] = nn.Linear(num_features, 32)
efficientnet.classifier.fc = nn.Linear(32,7)
efficientnet = efficientnet.to(device)

for param in efficientnet.features.parameters():
  param.requires_grad = False
for params in efficientnet.classifier.parameters():
  params.requires_grad = True
print(efficientnet.classifier)

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=32, bias=True)
  (fc): Linear(in_features=32, out_features=7, bias=True)
)


In [96]:
class Gaussian_blur():
  def __init__(self,mean,std):
    self.mean = mean
    self.std = std

  def __call__(self,img):
    img = np.array(img)/255.0
    h,w,c = img.shape
    noise = np.random.normal(self.mean,self.std,img.shape)
    img = np.clip(img+noise,0,1)
    img = np.uint8(img*255)
    img = img[:,:,:3]
    return Image.fromarray(img)

In [1]:
d = {'Crown gall(Agrobacterium tumefacins)':[['Remove infected plants, avoid injuring stems, and sterilize pruning tools.'],['Copper-based sprays, Agrobacterium radiobacter (a biological control agent)']],
     'Healthy' : [['Congrats'],['No further assistance needed']],
     'Rust(phragmidium species)':[['Prune infected parts, avoid wetting foliage, and use fungicides if necessary.'],['Copper-based sprays, sulfur fungicides, or mancozeb.']],
     'black rust':[['Remove infected leaves, improve air circulation, and use fungicides.'],['Neem oil, copper-based sprays, chlorothalonil, mancozeb, or myclobutanil.']],
     'botrysis blight':[['Remove dead flowers, ensure proper spacing, and use fungicides if needed.'],['Captan, thiophanate-methyl, or chlorothalonil.']],
     'phodosphera ponnosa':[['Ensure good airflow, avoid overhead watering, and apply fungicides.'],['Sulfur-based sprays, potassium bicarbonate, myclobutanil, or triforine.']],
     'roserosette':[['Remove and destroy infected plants; control mite populations.'],['Horticultural oils, insecticidal soap, or abamectin-based miticides.']]
    }


In [98]:
diseases_pth = os.path.join('/content/drive/MyDrive/Crop Survailance/Leaf Disease')

In [99]:
diseases = list()
for i in d.keys():
  diseases.append(i)
diseases

['Crown gall(Agrobacterium tumefacins)',
 'Healthy',
 'Rust(phragmidium species)',
 'black rust',
 'botrysis blight',
 'phodosphera ponnosa',
 'roserosette']

In [100]:
Transforms = transforms.Compose([transforms.Resize((224,224)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomVerticalFlip(),
                                 transforms.RandomRotation(20),
                                 transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                                 transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                                 Gaussian_blur(0,0.01),
                                 transforms.ToTensor()])
val_trans = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor()])

In [101]:
train_pth = ImageFolder(root = diseases_pth, transform = Transforms)

train_data = DataLoader(train_pth, batch_size = 32, shuffle = True)

In [102]:
train_pth

Dataset ImageFolder
    Number of datapoints: 142
    Root location: /content/drive/MyDrive/Crop Survailance/Leaf Disease
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               RandomRotation(degrees=[-20.0, 20.0], interpolation=nearest, expand=False, fill=0)
               RandomAffine(degrees=[0.0, 0.0], scale=(0.8, 1.2), shear=[-10.0, 10.0])
               ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=None)
               ToTensor()
           )

In [109]:
epochs = 64
efficientnet.train()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficientnet.parameters(), lr = 0.0001,weight_decay = 1e-3)

for epoch in range(epochs) :
  running_loss = 0
  for img,label in train_data:
    img, label = img.to(device),label.to(device)
    output = efficientnet(img)
    loss = criterion(output,label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_data)}')

Epoch 1/64, Loss: 1.131200337409973
Epoch 2/64, Loss: 1.0899667739868164
Epoch 3/64, Loss: 1.087049674987793
Epoch 4/64, Loss: 1.0954825401306152
Epoch 5/64, Loss: 1.053417479991913
Epoch 6/64, Loss: 1.0546252608299256
Epoch 7/64, Loss: 1.0643682718276977
Epoch 8/64, Loss: 1.0145792245864869
Epoch 9/64, Loss: 1.010298240184784
Epoch 10/64, Loss: 1.0052204132080078
Epoch 11/64, Loss: 0.9970182776451111
Epoch 12/64, Loss: 0.9656113266944886
Epoch 13/64, Loss: 0.9258787393569946
Epoch 14/64, Loss: 0.9424671053886413
Epoch 15/64, Loss: 0.9072110652923584
Epoch 16/64, Loss: 0.8819768071174622
Epoch 17/64, Loss: 0.9151532053947449
Epoch 18/64, Loss: 0.8916651368141174
Epoch 19/64, Loss: 0.8970477342605591
Epoch 20/64, Loss: 0.8623359799385071
Epoch 21/64, Loss: 0.913246214389801
Epoch 22/64, Loss: 0.8559035062789917
Epoch 23/64, Loss: 0.8511239647865295
Epoch 24/64, Loss: 0.8408726215362549
Epoch 25/64, Loss: 0.8066554069519043
Epoch 26/64, Loss: 0.7929654121398926
Epoch 27/64, Loss: 0.78677

In [112]:
img = Image.open('/content/drive/MyDrive/Crop Survailance/Leaf Disease/black rust/Screenshot_2025-03-09-22-13-47-49_40deb401b9ffe8e1df2f1cc5ba480b12.jpg')
img = val_trans(img)
#img = img.permute(1,2,0).numpy()
#plt.imshow(img)
img = img.unsqueeze(0)
img = img.to(device)
output = efficientnet(img)
pred = torch.argmax(output,1)
pred_disease = diseases[pred]
print("Index value : ",pred_disease,"Index : ",pred)
print(f"Suggestions : {d[pred_disease][0]} \n Pesticides Needed : {d[pred_disease][1]}")

Index value :  black rust Index :  tensor([3])
Suggestions : ['Remove infected leaves, improve air circulation, and use fungicides.'] 
 Pesticides Needed : ['Neem oil, copper-based sprays, chlorothalonil, mancozeb, or myclobutanil.']


In [15]:
cam = cv.VideoCapture(0)
if not cam.isOpened():
  print("Failed to open the camera")
  exit()

while True:
  ret,frame = cam.read()
  if not ret:
    print("Falied to initialize the camera")
    break

  img = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
  img = Image.fromarray(img)
  img = Transforms(img)
  img = img.unsqueeze(0)
  img = img.to(device)
  output = efficientnet(img)
  pred = torch.argmax(output,1)
  pred_disease = diseases[pred]
  cv.addText(frame,pred_disease,(10,30),cv.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2,cv.LINE_AA)

  cv.imshow('Disease',img)
  if cv.waitKey(1) & 0xFF == ord('q'):
    break

cam.release()
cv.destroyAllWindows()

Failed to open the camera
Falied to initialize the camera


In [38]:
train_pth.class_to_idx

{'Crown gall(Agrobacterium tumefacins)': 0,
 'Healthy': 1,
 'Rust(phragmidium species)': 2,
 'black rust': 3,
 'botrysis blight': 4,
 'phodosphera ponnosa': 5,
 'roserosette': 6}

In [113]:
torch.save(efficientnet.state_dict(),'efficientnet_weights_64.pth')